In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.colors import BoundaryNorm
from netCDF4 import Dataset
import cartopy.crs as ccrs

In [ ]:
#"""
region     = 'CONUS'
ilon_start =  45
ilon_end   = 120
ilat_start = 110
ilat_end   = 150
#"""

isl        = 1
YYYY_list  = [2012];
nYYYY      = len(YYYY_list)
plt.rcParams.update({'font.size': 22})
nx         = ilon_end-ilon_start + 1;
ny         = ilat_end-ilat_start + 1;


In [ ]:
WWLLN_dir       = '/home/disk/eos12/wycheng/data/US/WWLLN/'
ERA5_dir        = '/home/disk/eos12/wycheng/data/US/ERA5/'
TRMM_dir        = '/home/disk/eos12/wycheng/data/US/TRMM/'
island_dir      = '/home/disk/eos9/wycheng/LightningMachineLearning/data/cgdata/island/'

island_fname    = 'island_1deg.nc'
ncin_island     = Dataset(island_dir+island_fname,'r')
island_in       = ncin_island.variables['island'][ilat_start:ilat_end,ilon_start:ilon_end]

nYYYY           = np.shape(YYYY_list)[0]

for iYYYY in range(nYYYY):
    YYYY = YYYY_list[iYYYY];

    WWLLN_F_fname   = 'WWLLN_'+str(YYYY)+'_F_cg_1deg3hr_US.nc'
    ERA5_cape_fname = 'ERA5_'+str(YYYY)+'_cape_cg_1deg3hr_US.nc'
    TRMM_pcp_fname  = 'TRMM_'+str(YYYY)+'_pcp_cg_1deg3hr_US.nc'

    ncin_F          = Dataset(WWLLN_dir+WWLLN_F_fname,'r')
    ncin_cape       = Dataset(ERA5_dir+ERA5_cape_fname,'r')
    ncin_pcp        = Dataset(TRMM_dir+TRMM_pcp_fname,'r')

    
    if (iYYYY==0):
        F_in        = ncin_F.variables['F'][:,:,:]
        cape_in     = ncin_cape.variables['cape'][:,:,:]
        pcp_in      = ncin_pcp.variables['pcp'][:,:,:]

    else:
        F_in        = np.append(F_in,ncin_F.variables['F'][:,:,:],axis=0)
        cape_in     = np.append(cape_in,ncin_cape.variables['cape'][:,:,:],axis=0)
        pcp_in      = np.append(pcp_in,ncin_pcp.variables['pcp'][:,:,:],axis=0)

F_in        = F_in * (1/((111.19492664455873)**2)) * (365.25*8) # turn unit into [km-2 yr-1]
sqrtcape_in = cape_in ** 0.5;

island_in3d = np.broadcast_to(island_in, F_in.shape)

In [ ]:
print(island_in3d.shape)
print(F_in.shape)
print(cape_in.shape)
print(sqrtcape_in.shape)
print(pcp_in.shape)

In [ ]:
mask_island   = island_in3d==1;
print(mask_island.shape)
mask_normal   = 1.0*mask_island;
mask_normal   = np.where(mask_island==0, np.nan, mask_island);
print(mask_normal.shape)

In [ ]:
clevs0          = np.arange(0,50,20)

llcrnrlon       = -135
llcrnrlat       =   20
urcrnrlon       =  -60
urcrnrlat       =   60
coastlinewidth  =    3.
coastlinecolor  = 'gray'
lat             = np.linspace(  20, 60,  ny);#np.linspace( -89.5, 89.5,ny);
lon             = np.linspace(-135,-60,  nx);#np.linspace(-179.5,179.5,nx);
lons, lats      = np.meshgrid(lon[:], lat[:]);#np.meshgrid(lon[ilon_start:ilon_end], lat[ilat_start:ilat_end])

In [ ]:
plt.rcParams.update({'font.size': 48})
figsize = (40,20)
cmap = plt.get_cmap('jet')

In [ ]:
def plot_map(figsize,var,lons,lats,cmap,varminmax,title,unit):
    
    fig   = plt.figure(figsize=figsize)
    ax    = plt.axes(projection=ccrs.PlateCarree())

    pcm   = plt.pcolormesh(lons, lats, var, cmap=cmap, norm=mcolors.PowerNorm(1.0))

    gl    = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top   = False
    gl.ylabels_right = False
    ax.coastlines()

    plt.clim(varminmax[0],varminmax[1])
    plt.title(title)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=1, axes_class=plt.Axes)
    cbar = plt.colorbar(pcm, cax=cax,extend='max')
    plt.text(varminmax[0],varminmax[1]*1.08,unit)

    plt.show()

# Normal Thunderstorm

In [ ]:
array = np.nanmean(pcp_in*mask_normal,axis=0)[:,:]
varminmax = [0,0.2]
unit = '[mm hr$^{-1}$]'
title = 'Precip'
plot_map(figsize,array,lons,lats,cmap,varminmax,title,unit)

In [ ]:
array = np.nanmean(cape_in*mask_normal,axis=0)[:,:]
varminmax = [0,800]
unit = '[J kg$^{-1}$]'
title = 'CAPE'
plot_map(figsize,array,lons,lats,cmap,varminmax,title,unit)

In [ ]:
F_normal = np.sum((F_in>0)*mask_normal,axis=0)
varminmax = [0,500]
unit = '[#]'
title = 'Lightning events'
plot_map(figsize,F_normal,lons,lats,cmap,varminmax,title,unit)

In [ ]:
array = np.nanmean(cape_in*pcp_in*mask_normal,axis=0)[:,:]
varminmax = [0,200]
unit = ''
title = 'CAPE x pcp'
plot_map(figsize,array,lons,lats,cmap,varminmax,title,unit)

# Dry Thunderstorm

In [ ]:
pcp_thrs      = 2.5/3
mask_pcpthrs  = pcp_in<pcp_thrs;
mask_dt       = 1.0*mask_pcpthrs*mask_island;
mask_dt       = np.where(mask_dt==0, np.nan, mask_dt);
print(mask_dt.shape)

In [ ]:
array = np.nansum((F_in>0)*mask_dt,axis=0)[:,:]
F_dry = array

varminmax = [0,200]
unit = '[#]'
title = 'Dry thunderstorm events'
plot_map(figsize,F_dry,lons,lats,cmap,varminmax,title,unit)

In [ ]:
array = 100*np.where(F_normal>0, F_dry/F_normal , np.nan)

varminmax = [0,100]
unit = '[%]'
title = 'Dry thunderstorm fraction'
plot_map(figsize,array,lons,lats,cmap,varminmax,title,unit)